## I built a model from scratch because I got mad at Gurobi (Model Explanation) ##

Here's how the model works, given an unordered list of cities and a complete weighted graph.

To construct a full tour the algorithm first sorts the list in the following way:

Given a list of nodes:
1) The first city in the list is saved as a reference
2) All other cities are sorted in order of how far away they are from the first city

After the list is sorted the list is split into even and odd indices, then the odd indices list is mirrored in order, then the odd indices list is appended to the even list again. Then afterwards the first element of the reappended list is put on the end to complete the cycle.

E.g. for the list of nodes (A,B,C,D), assuming the list is sorted as described above with A as the start of the tour:
1) It's split into (A,C), the even list, and (B,D), the odd list
2) (B,D) becomes (D,B)
3) Then the lists are reappened to make (A,C,D,B)
4) Then A is appended on to the end to make (A,C,D,B,A)

The list output by this process is assumed to be a very strong approximation of the optimal order of nodes.

Once the sorted touring order is obtained the model uses a greedy graph search algorithm. When constructing a route between two points the model finds the immediate best next node by seeing which middle node is the closest to the target node. The model uses this process to find the assumed shortest routes between cities in the sorted list in order (e.g. A->C, then C->D, etc.) until all adjacent nodes are connected by a route. During tour construction a record is kept of which cities have already been visited to make sure that there are no repeat locations.


In [92]:
#### Cell Purpose: Set up data and coefficients ####

from fileinput import close
import pandas as pd
import folium
import itertools as iter
import random

#### Make Data ####

#open city files
city_names_text = open(r'../OIE project/Data/USCA312_labels.txt', 'r')
city_distances_text = open(r'../OIE project/Data/USCA312_distances.txt', 'r')


#put lines of city_names_Text in a list
city_names_list = [line.strip() for line in city_names_text]
#cut out the filler text at the beginning of the txt file
city_names_list = city_names_list[15:]

#put lines of city_names_Text in a list
city_dist_list = [line.strip() for line in city_distances_text]
#cut out the filler text at the beginning of the txt file
city_dist_list = city_dist_list[7:]
#split the individual numbers into their own elements, but they're split into sublists by line
city_dist_list = [line.split() for line in city_dist_list]
#merge all the sublists
city_dist_list = [element #return element, append to list
                    for sublist in city_dist_list #for each sublist
                        for element in sublist] #for each element in each sublist
#split the main list back up into chunks for each city
city_dist_list = [city_dist_list[city_index*len(city_names_list)
                                 :
                                 (city_index+1)*len(city_names_list)] #return a range that is a block of 312 cities
                        for city_index in range(len(city_names_list))] #for each city in city_names_list

#Create the dataframe
city_dist_df = pd.DataFrame(city_dist_list, #data is the distances list
                            index=city_names_list, #row and column indices are the city list
                            columns=city_names_list)

#quick function for defining distances
def distance(city1, city2):
    return int(city_dist_df[city1][city2])

#ok python why are you making me do this at the end of every block
#city_names_list = list(city_dist_df.columns)

city_names_text.close()
city_distances_text.close()

#Coach bus drivers can only drive 10 hours per day legally
#Rule of thumb in the industry is that 600 miles is the cap for a legal drive, see source below
# https://mcatransportation.com/legal-driving-hours/#:~:text=The%20basic%20rule%20of%20thumb,also%20make%20the%20trip%20safe.
#Driving range can be altered for future testing
driver_range = 600

In [93]:
#### Cell Purpose: Create a function to find the next best edge to the target ####

#iterates over all possible edges, finds edge that makes the most progress towards the target city
def best_edge(start, end, forbidden_cities=[]):
    encumbent_dist = 10000000
    encumbent = ()
    #for every city
    for city in city_names_list:
        #If the city being checked is not the starting city, meets range criteria, 
        #and is not blacklisted for other reasons
        if (city!=start 
            and 
            distance(start, city) <= driver_range
            and
            city not in forbidden_cities):
            #if, given a point between a and b, 
            if distance(city, end) < encumbent_dist:
                encumbent = (start, city)
                encumbent_dist = distance(city, end)
                
    return encumbent

In [94]:
#### Cell Purpose: Create a function to construct the best route between two points ####

#finds the best route between two cities, with an option of a list of cities to forbid
def best_route(start, end, forbidden_cities=[]):
    #store the route
    route = []
    #keep track of the last node visited
    last_node = start
    
    #a counter to make sure there's no infinite loop
    error_counter = 0

    #keep going until the route reaches the target destination
    while(last_node != end):
        #find the next edge in the route
        new_edge = best_edge(last_node, end, forbidden_cities)
        #add the edge to the route
        route.append(new_edge)
        #update the last_node
        last_node = new_edge[1]

        error_counter += 1

        #if the error counter is higher than the maximum route length the solution is infeasible
        if error_counter > len(city_names_list):
            raise Exception('Infeasible Solution')
        
    
    return route


In [95]:
#### Cell Purpose: Create a function to turn a list of edges into a list of the nodes visited ####

#function to flatten tuples into a list of cities
def flatten_route(tuple_route):
    #put the first element of the first tuple in the flattened list
    flat = [tuple_route[0][0]]
    #put the last element of every tuple in the flattened list
    for tup in tuple_route:
        flat.append(tup[1])
    
    return flat


In [96]:
#### Cell Purpose: Create a function to sort the unordered touring route ####

#sorts a list of cities in an efficient tour order
def sort_tour(touring_list):

    #method to sort all cities by distance from the first city in the touring list
    def sort_by_distance(city_list):
        #keep the first city stored
        first = city_list[0]

        #storing the distances and city labels together in tuples
        tuple_list = []

        #iterate over all cities
        for city in city_list:
            #make the tuple, add tuple to unsorted list
            tup = (city, distance(first, city))
            tuple_list.append(tup)
        
        #sort tuple list by distances, the things indexed at 1 in each tuple
        tuple_list.sort(key=lambda tup: tup[1])
        
        return tuple_list

    #sort the touring list
    sorted_list = sort_by_distance(touring_list)
    #remove all the distances so only the city labels remain
    sorted_list = [tup[0] for tup in sorted_list ]

    #split the sorted list into the odd indexed and even indexed elements
    #Here's why:
    #   Given a list of cities sorted by distance from A:
    #   (A, B, C, D, E, F, G)
    #   The route to follow for an efficient cycle would be
    #   A -> C -> E -> G -> F -> D -> B -> A
    #   You need to save a route back from the furthest point in the cycle
    #   This is most easily accomplished by just alternating indices,
    #   but iterating over the odd indices in reverse
    odd_list = []
    even_list = []
    for i in range(0, len(sorted_list)):
        if i % 2:
            even_list.append(sorted_list[i])
        else :
            odd_list.append(sorted_list[i])
    
    #flip the odd list (the one that doesn't contain element number 1) around
    odd_list.reverse()
    #then append the odd_list's elements to the even list
    for city in odd_list:
        even_list.append(city)

    return even_list

In [97]:
#### Cell Purpose: Create a function to find the best touring route given the sorted list ####

#finds the best tour over a pre-ordered list of cities
#uses a greedy graph search
#the route has no repeated destinations
def best_tour(city_list):

    #because the tour cycles, append the first city onto the end of the list
    first = city_list[0]
    city_list2 = city_list.append(first)

    #list to store the output
    tour = []
    #list to remember what cities not to use
    forbidden_cities = []

    #iterate over every city required for the tour
    #stop one city short to prevent an out of bounds error
    for city_index in range(0, len(city_list)-1):

        #find the best route between the current city and the following city
        route = best_route(city_list[city_index], city_list[city_index+1], forbidden_cities)
        #add all the edges to the tour
        for edge in route:
            tour.append(edge)

        ## all used cities are not allowed to be reused ##
        #flatten route of tour
        forbidden_cities = flatten_route(tour)
        #leave all the required cities out of the forbidden cities list
        forbidden_cities = [city for city in flatten_route(route) if city not in city_list]
    
    return tour
    

In [98]:
#### Cell Purpose: Create a function to find the best touring route given an unsorted list ####

#wrapper method to make a tour from an unsorted city list
def find_best_tour(city_list):
    sorted = sort_tour(city_list)
    best = best_tour(sorted)
    return best

In [99]:
#### Cell Purpose: Create a function to calculate the distance traveled over a tour ####

#method to evaluate tour distance when tour is still a list of edges
def find_tour_distance(edge_list):
    #variable to hold the total distance
    total = 0
    #for each edge traversed
    for edge in edge_list:
        #how far did you go?
        dist = distance(edge[0], edge[1])
        #add the edge distance to the total
        total += dist
    return total


In [100]:
#### Cell Purpose: User inputs a test route and solves test route ####

#The size of the random sample, change this as you like
sample_size = 7
#generate test list
test_list = random.sample(city_names_list, sample_size)

#You can put in your own list as long as you make sure the list is still named test_list

#find the best tour
#if this block throws an error your test list is infeasible
tour = find_best_tour(test_list)

In [101]:
#### Cell Purpose: Create helper methods for the map display ####

## Note: The points are all a little bit off of the actual cities because all the lat/long values are rounded

#data for all the lats and longs of the cities
city_coords_df = pd.read_csv('../OIE project/Data/USCA312_coords.csv')

#make a list of the coordinates of all cities in the route
def make_points_list(finished_tour):
    #list is for the polyline
    points_list = []
    for city in finished_tour:
        city_index = city_names_list.index(city)
        #store the lat and long of the city in both list and dict form
        points_list.append((city_coords_df.iloc[city_index]['Latitude'],
                            city_coords_df.iloc[city_index]['Longitude']))
    return points_list

#make a dict {city:distance} of the coordinates of all cities in the route
def make_points_dict(finished_tour):
    #dict is for the markers and their popups
    points_dict = {}
    for city in finished_tour:
        city_index = city_names_list.index(city)
        #store the lat and long of the city in both list and dict form   
        points_dict[city] = (city_coords_df.iloc[city_index]['Latitude'],
                            city_coords_df.iloc[city_index]['Longitude'])
    return points_dict

In [102]:
#### Cell Purpose: Generate data visualizations  ####

#create the route to plot
route = flatten_route(tour)
#the list of mandatory cities to mark
mandatory_cities = test_list

map = folium.Map(location=[40,-95], zoom_start = 4)

#for the markers of every city in the route and their popups
route_points_dict = make_points_dict(route)
#for the markers of the mandatory cities
mandatory_points_dict = make_points_dict(mandatory_cities)

#remove duplicates between route_dict and mandatory_dict
for key in mandatory_points_dict:
  del route_points_dict[key]

#list is for the polyline
points_list = make_points_list(route)

#create the line for the tour route, and all of the optional city markers
for key in route_points_dict:
  folium.PolyLine(points_list).add_to(map)
  folium.Marker(route_points_dict[key], popup=key).add_to(map)

#create the mandatory city markers
for key in mandatory_points_dict:
  folium.Marker(mandatory_points_dict[key], popup=key, icon=folium.Icon(icon="cloud")).add_to(map)

In [103]:
#### Cell Purpose: Show data visualizations ####

## map markers legend:
## circle icon = optional city
## cloud icon = mandatory tour location

print('Mandatory cities:', test_list, '\n')
print('Route taken:', flatten_route(tour), '\n')
print('Total distance traveled:', find_tour_distance(tour), 'miles')
map

Mandatory cities: ['Winnipeg, MB', 'Kansas City, MO', 'Concord, NH', 'San Jose, CA', 'Wichita, KS', 'Austin, TX', 'Meriden, CT'] 

Route taken: ['Kansas City, MO', 'Jackson, MI', 'Pittsfield, MA', 'Concord, NH', 'Zanesville, OH', 'Memphis, TN', 'Austin, TX', 'El Paso, TX', 'Las Vegas, NV', 'San Jose, CA', 'Ogden, UT', 'Rapid City, SD', 'Eau Claire, WI', 'Erie, PA', 'Meriden, CT', 'Lima, OH', 'Kansas City, KS', 'Wichita, KS', 'Saint Cloud, MN', 'Winnipeg, MB', 'Des Moines, IA', 'Kansas City, MO'] 

Total distance traveled: 10078 miles
